In [5]:
import pandas as pd
from pathlib import Path

# 1. 需要处理的数据集路径
file_map = {
    'ETTh1': './datasets/ETTh1.csv',
    'ETTh2': './datasets/ETTh2.csv',
    'Traffic': './datasets/Traffic.csv',
    'Electricity': './datasets/Electricity.csv',
    'Weather': './datasets/Weather.csv',
    'ETTm1': './datasets/ETTm1.csv',
    'ETTm2': './datasets/ETTm2.csv'
}

# 2. 每个文件对应的时间列名（按实际情况改）
time_col_map = {
    'ETTh1': 'date',
    'ETTh2': 'date',
    'ETTm1': 'date',
    'ETTm2': 'date',
    'Traffic': 'date',      # 如果 Traffic 里叫别的名字，改这里
    'Electricity': 'date',  # 同上
    'Weather': 'date'       # 同上
}

# 3. 统一截断函数
def clip_one_year(df: pd.DataFrame, time_col: str) -> pd.DataFrame:
    """只保留最近一年的数据"""
    if time_col not in df.columns:
        raise ValueError(f'列 {time_col} 不存在，请检查 time_col_map 设置')
    # 转时间戳
    df[time_col] = pd.to_datetime(df[time_col])
    # 计算截断边界
    max_day = df[time_col].max()
    cutoff = max_day - pd.Timedelta(days=365)
    # 截断并复位索引
    return df[df[time_col] >= cutoff].reset_index(drop=True)

# 4. 批量读取 + 截断
dfs = {}
for name, path in file_map.items():
    if not Path(path).exists():
        print(f'⚠️ 文件不存在，跳过：{path}')
        continue
    df = pd.read_csv(path)
    dfs[name] = clip_one_year(df, time_col_map[name])
    print(f'{name:10s} 原始条数 {len(df):6d} -> 截断后条数 {len(dfs[name]):6d}')

# 5. 把结果重新赋给你原来用的变量名
ETTh1 = dfs.get('ETTh1')
ETTh2 = dfs.get('ETTh2')
Traffic = dfs.get('Traffic')
Electricity = dfs.get('Electricity')
Weather = dfs.get('Weather')
ETTm1 = dfs.get('ETTm1')
ETTm2 = dfs.get('ETTm2')

ETTh1      原始条数  17420 -> 截断后条数   8761
ETTh2      原始条数  17420 -> 截断后条数   8761
Traffic    原始条数  17544 -> 截断后条数   8761
Electricity 原始条数  26304 -> 截断后条数   8761
Weather    原始条数  52696 -> 截断后条数  52553
ETTm1      原始条数  69680 -> 截断后条数  35041
ETTm2      原始条数  69680 -> 截断后条数  35041


In [1]:
import pandas as pd
from pathlib import Path
import os

# 1. 原始文件列表
src_files = [
    './datasets/ETTh1.csv',
    './datasets/ETTh2.csv',
    './datasets/Traffic.csv',
    './datasets/Electricity.csv',
    './datasets/Weather.csv',
    './datasets/ETTm1.csv',
    './datasets/ETTm2.csv'
]

# 2. 每个文件对应的时间列（按实际修改）
time_cols = {
    'ETTh1.csv': 'date',
    'ETTh2.csv': 'date',
    'ETTm1.csv': 'date',
    'ETTm2.csv': 'date',
    'Traffic.csv': 'date',      # 如果叫 OT 就改成 'OT'
    'Electricity.csv': 'date',
    'Weather.csv': 'date'
}

# 3. 输出目录
out_dir = Path('./datasets/one_year')
out_dir.mkdir(exist_ok=True, parents=True)

# 4. 截断并保存函数
def clip_and_save(src_path: str):
    fname = os.path.basename(src_path)          # 例如 'ETTh1.csv'
    df = pd.read_csv(src_path)

    # 取时间列
    tcol = time_cols.get(fname)
    if tcol is None or tcol not in df.columns:
        raise ValueError(f'请确认 {fname} 的时间列名')

    # 转 datetime
    df[tcol] = pd.to_datetime(df[tcol])
    cutoff = df[tcol].max() - pd.Timedelta(days=365)
    df_1y = df[df[tcol] >= cutoff].reset_index(drop=True)

    # 构造新路径
    new_path = out_dir / fname.replace('.csv', '.csv')
    df_1y.to_csv(new_path, index=False)
    print(f'{fname:15s}  {len(df):6d} -> {len(df_1y):6d}  已保存到 {new_path}')

# 5. 批量执行
for f in src_files:
    if Path(f).is_file():
        clip_and_save(f)
    else:
        print(f'⚠️ 文件不存在：{f}')

ETTh1.csv         17420 ->   8761  已保存到 datasets\one_year\ETTh1.csv
ETTh2.csv         17420 ->   8761  已保存到 datasets\one_year\ETTh2.csv
Traffic.csv       17544 ->   8761  已保存到 datasets\one_year\Traffic.csv
Electricity.csv   26304 ->   8761  已保存到 datasets\one_year\Electricity.csv
Weather.csv       52696 ->  52553  已保存到 datasets\one_year\Weather.csv
ETTm1.csv         69680 ->  35041  已保存到 datasets\one_year\ETTm1.csv
ETTm2.csv         69680 ->  35041  已保存到 datasets\one_year\ETTm2.csv
